Let us first do some dimension reduction of the word vector.

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
from utils.get_settings import parse

settings = parse("./utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'}}

In [3]:
train_data = pd.read_pickle(settings["train"]["pickle_path"][1:])
test_data = pd.read_pickle(settings["test"]["pickle_path"][1:])
valid_data = pd.read_pickle(settings["valid"]["pickle_path"][1:])

In [4]:
train_vector_ = np.concatenate(train_data.vector_bert_1024,axis = 0)
valid_vector_ = np.concatenate(train_data.vector_bert_1024,axis = 0)
test_vector_ = np.concatenate(train_data.vector_bert_1024,axis = 0)

In [5]:
vector = np.concatenate([train_vector_,valid_vector_,test_vector_],axis = 0)

In [6]:
vector.shape

(669522, 1024)

In [7]:
import torch
import torch.nn as nn
from utils.AE import *


batch_size = 256
learning_rate = 1e-3
steps = 10000

np.random.seed(0)
torch.manual_seed(0)

In [8]:
mean_vector_1 = torch.Tensor(vector.mean(0,keepdims = True)).cuda()

In [9]:
PPA_1 = autoencoder(1024,4)
PPA_1.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PPA_1.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,vector.shape[0] - batch_size+1)
    data_train = torch.Tensor(vector[pos:pos+batch_size,:]).cuda()-mean_vector_1
    # ===================forward=====================
    data_reconstruction, _,_ = PPA_1(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PPA_1.state_dict(), './dimension_reduction/PPA1_1024.pth')

steps [1/10000], loss:0.3599
steps [2/10000], loss:0.4020
steps [3/10000], loss:0.3692
steps [4/10000], loss:0.3721
steps [5/10000], loss:0.3715
steps [6/10000], loss:0.3656
steps [7/10000], loss:0.3741
steps [8/10000], loss:0.3877
steps [9/10000], loss:0.3669
steps [10/10000], loss:0.3998
steps [11/10000], loss:0.3492
steps [12/10000], loss:0.3831
steps [13/10000], loss:0.3816
steps [14/10000], loss:0.3816
steps [15/10000], loss:0.4030
steps [16/10000], loss:0.3869
steps [17/10000], loss:0.3750
steps [18/10000], loss:0.3446
steps [19/10000], loss:0.3512
steps [20/10000], loss:0.3629
steps [21/10000], loss:0.3652
steps [22/10000], loss:0.3803
steps [23/10000], loss:0.3473
steps [24/10000], loss:0.4036
steps [25/10000], loss:0.3659
steps [26/10000], loss:0.3563
steps [27/10000], loss:0.3520
steps [28/10000], loss:0.3337
steps [29/10000], loss:0.3564
steps [30/10000], loss:0.3348
steps [31/10000], loss:0.3498
steps [32/10000], loss:0.3446
steps [33/10000], loss:0.3533
steps [34/10000], l

steps [314/10000], loss:0.3458
steps [315/10000], loss:0.3106
steps [316/10000], loss:0.3221
steps [317/10000], loss:0.3110
steps [318/10000], loss:0.3380
steps [319/10000], loss:0.3111
steps [320/10000], loss:0.3387
steps [321/10000], loss:0.3406
steps [322/10000], loss:0.3120
steps [323/10000], loss:0.3347
steps [324/10000], loss:0.3347
steps [325/10000], loss:0.3295
steps [326/10000], loss:0.3379
steps [327/10000], loss:0.3081
steps [328/10000], loss:0.3292
steps [329/10000], loss:0.3138
steps [330/10000], loss:0.3128
steps [331/10000], loss:0.3358
steps [332/10000], loss:0.3138
steps [333/10000], loss:0.3163
steps [334/10000], loss:0.3435
steps [335/10000], loss:0.3320
steps [336/10000], loss:0.3381
steps [337/10000], loss:0.3161
steps [338/10000], loss:0.3195
steps [339/10000], loss:0.3357
steps [340/10000], loss:0.3067
steps [341/10000], loss:0.3397
steps [342/10000], loss:0.3504
steps [343/10000], loss:0.3526
steps [344/10000], loss:0.3152
steps [345/10000], loss:0.3465
steps [3

steps [664/10000], loss:0.3180
steps [665/10000], loss:0.3365
steps [666/10000], loss:0.2778
steps [667/10000], loss:0.3088
steps [668/10000], loss:0.2532
steps [669/10000], loss:0.2849
steps [670/10000], loss:0.3000
steps [671/10000], loss:0.3010
steps [672/10000], loss:0.2680
steps [673/10000], loss:0.2820
steps [674/10000], loss:0.3065
steps [675/10000], loss:0.2829
steps [676/10000], loss:0.3015
steps [677/10000], loss:0.3119
steps [678/10000], loss:0.2906
steps [679/10000], loss:0.2994
steps [680/10000], loss:0.2824
steps [681/10000], loss:0.2881
steps [682/10000], loss:0.3100
steps [683/10000], loss:0.2665
steps [684/10000], loss:0.2898
steps [685/10000], loss:0.2878
steps [686/10000], loss:0.2764
steps [687/10000], loss:0.2752
steps [688/10000], loss:0.2797
steps [689/10000], loss:0.2789
steps [690/10000], loss:0.3126
steps [691/10000], loss:0.3157
steps [692/10000], loss:0.3156
steps [693/10000], loss:0.3165
steps [694/10000], loss:0.3117
steps [695/10000], loss:0.3138
steps [6

steps [1007/10000], loss:0.3097
steps [1008/10000], loss:0.2774
steps [1009/10000], loss:0.3205
steps [1010/10000], loss:0.2791
steps [1011/10000], loss:0.3058
steps [1012/10000], loss:0.2730
steps [1013/10000], loss:0.2472
steps [1014/10000], loss:0.2795
steps [1015/10000], loss:0.2929
steps [1016/10000], loss:0.3135
steps [1017/10000], loss:0.3276
steps [1018/10000], loss:0.2907
steps [1019/10000], loss:0.3149
steps [1020/10000], loss:0.2865
steps [1021/10000], loss:0.2706
steps [1022/10000], loss:0.2885
steps [1023/10000], loss:0.2890
steps [1024/10000], loss:0.2740
steps [1025/10000], loss:0.2641
steps [1026/10000], loss:0.2974
steps [1027/10000], loss:0.3154
steps [1028/10000], loss:0.2975
steps [1029/10000], loss:0.2856
steps [1030/10000], loss:0.2932
steps [1031/10000], loss:0.2767
steps [1032/10000], loss:0.2850
steps [1033/10000], loss:0.2999
steps [1034/10000], loss:0.2558
steps [1035/10000], loss:0.2873
steps [1036/10000], loss:0.2787
steps [1037/10000], loss:0.3032
steps [1

steps [1359/10000], loss:0.3091
steps [1360/10000], loss:0.2695
steps [1361/10000], loss:0.2572
steps [1362/10000], loss:0.2936
steps [1363/10000], loss:0.3192
steps [1364/10000], loss:0.2931
steps [1365/10000], loss:0.2708
steps [1366/10000], loss:0.2645
steps [1367/10000], loss:0.2895
steps [1368/10000], loss:0.3014
steps [1369/10000], loss:0.2764
steps [1370/10000], loss:0.2749
steps [1371/10000], loss:0.3224
steps [1372/10000], loss:0.3351
steps [1373/10000], loss:0.2847
steps [1374/10000], loss:0.2761
steps [1375/10000], loss:0.2817
steps [1376/10000], loss:0.2942
steps [1377/10000], loss:0.2906
steps [1378/10000], loss:0.2856
steps [1379/10000], loss:0.2699
steps [1380/10000], loss:0.3275
steps [1381/10000], loss:0.2787
steps [1382/10000], loss:0.3063
steps [1383/10000], loss:0.2838
steps [1384/10000], loss:0.2679
steps [1385/10000], loss:0.2750
steps [1386/10000], loss:0.2758
steps [1387/10000], loss:0.3330
steps [1388/10000], loss:0.2567
steps [1389/10000], loss:0.3039
steps [1

...

steps [8240/10000], loss:0.2962
steps [8241/10000], loss:0.2821
steps [8242/10000], loss:0.2869
steps [8243/10000], loss:0.2585
steps [8244/10000], loss:0.2777
steps [8245/10000], loss:0.2971
steps [8246/10000], loss:0.2686
steps [8247/10000], loss:0.2725
steps [8248/10000], loss:0.2795
steps [8249/10000], loss:0.2868
steps [8250/10000], loss:0.3047
steps [8251/10000], loss:0.3019
steps [8252/10000], loss:0.2813
steps [8253/10000], loss:0.2626
steps [8254/10000], loss:0.3105
steps [8255/10000], loss:0.3172
steps [8256/10000], loss:0.3569
steps [8257/10000], loss:0.2606
steps [8258/10000], loss:0.2787
steps [8259/10000], loss:0.3011
steps [8260/10000], loss:0.2949
steps [8261/10000], loss:0.2943
steps [8262/10000], loss:0.2685
steps [8263/10000], loss:0.2967
steps [8264/10000], loss:0.2982
steps [8265/10000], loss:0.2731
steps [8266/10000], loss:0.2786
steps [8267/10000], loss:0.3233
steps [8268/10000], loss:0.3044
steps [8269/10000], loss:0.2720
steps [8270/10000], loss:0.3011
steps [8

steps [8622/10000], loss:0.2908
steps [8623/10000], loss:0.2660
steps [8624/10000], loss:0.2977
steps [8625/10000], loss:0.2858
steps [8626/10000], loss:0.3429
steps [8627/10000], loss:0.2865
steps [8628/10000], loss:0.2773
steps [8629/10000], loss:0.2912
steps [8630/10000], loss:0.2950
steps [8631/10000], loss:0.2700
steps [8632/10000], loss:0.2955
steps [8633/10000], loss:0.3030
steps [8634/10000], loss:0.2937
steps [8635/10000], loss:0.2776
steps [8636/10000], loss:0.3020
steps [8637/10000], loss:0.2886
steps [8638/10000], loss:0.2743
steps [8639/10000], loss:0.2865
steps [8640/10000], loss:0.3010
steps [8641/10000], loss:0.2603
steps [8642/10000], loss:0.3574
steps [8643/10000], loss:0.2766
steps [8644/10000], loss:0.2494
steps [8645/10000], loss:0.3251
steps [8646/10000], loss:0.2660
steps [8647/10000], loss:0.3163
steps [8648/10000], loss:0.2648
steps [8649/10000], loss:0.2732
steps [8650/10000], loss:0.2680
steps [8651/10000], loss:0.2720
steps [8652/10000], loss:0.2631
steps [8

steps [8985/10000], loss:0.3022
steps [8986/10000], loss:0.3107
steps [8987/10000], loss:0.2652
steps [8988/10000], loss:0.2919
steps [8989/10000], loss:0.2938
steps [8990/10000], loss:0.2796
steps [8991/10000], loss:0.2599
steps [8992/10000], loss:0.2709
steps [8993/10000], loss:0.2628
steps [8994/10000], loss:0.2801
steps [8995/10000], loss:0.3234
steps [8996/10000], loss:0.2501
steps [8997/10000], loss:0.2851
steps [8998/10000], loss:0.2646
steps [8999/10000], loss:0.2759
steps [9000/10000], loss:0.2677
steps [9001/10000], loss:0.2592
steps [9002/10000], loss:0.2625
steps [9003/10000], loss:0.2897
steps [9004/10000], loss:0.3003
steps [9005/10000], loss:0.2698
steps [9006/10000], loss:0.2807
steps [9007/10000], loss:0.3045
steps [9008/10000], loss:0.2693
steps [9009/10000], loss:0.3207
steps [9010/10000], loss:0.2833
steps [9011/10000], loss:0.2466
steps [9012/10000], loss:0.2839
steps [9013/10000], loss:0.3331
steps [9014/10000], loss:0.2779
steps [9015/10000], loss:0.2878
steps [9

steps [9355/10000], loss:0.2691
steps [9356/10000], loss:0.2691
steps [9357/10000], loss:0.2658
steps [9358/10000], loss:0.2994
steps [9359/10000], loss:0.2754
steps [9360/10000], loss:0.3070
steps [9361/10000], loss:0.2540
steps [9362/10000], loss:0.3091
steps [9363/10000], loss:0.2608
steps [9364/10000], loss:0.2660
steps [9365/10000], loss:0.2802
steps [9366/10000], loss:0.2915
steps [9367/10000], loss:0.2710
steps [9368/10000], loss:0.2731
steps [9369/10000], loss:0.2658
steps [9370/10000], loss:0.2929
steps [9371/10000], loss:0.2879
steps [9372/10000], loss:0.2722
steps [9373/10000], loss:0.2939
steps [9374/10000], loss:0.2930
steps [9375/10000], loss:0.3018
steps [9376/10000], loss:0.3108
steps [9377/10000], loss:0.2771
steps [9378/10000], loss:0.2842
steps [9379/10000], loss:0.2732
steps [9380/10000], loss:0.2742
steps [9381/10000], loss:0.2921
steps [9382/10000], loss:0.2790
steps [9383/10000], loss:0.2684
steps [9384/10000], loss:0.2964
steps [9385/10000], loss:0.3223
steps [9

steps [9725/10000], loss:0.2890
steps [9726/10000], loss:0.2674
steps [9727/10000], loss:0.2998
steps [9728/10000], loss:0.2761
steps [9729/10000], loss:0.2718
steps [9730/10000], loss:0.2792
steps [9731/10000], loss:0.2822
steps [9732/10000], loss:0.2673
steps [9733/10000], loss:0.2737
steps [9734/10000], loss:0.2822
steps [9735/10000], loss:0.2695
steps [9736/10000], loss:0.2926
steps [9737/10000], loss:0.2978
steps [9738/10000], loss:0.2817
steps [9739/10000], loss:0.2769
steps [9740/10000], loss:0.3383
steps [9741/10000], loss:0.2908
steps [9742/10000], loss:0.2962
steps [9743/10000], loss:0.2606
steps [9744/10000], loss:0.3501
steps [9745/10000], loss:0.2839
steps [9746/10000], loss:0.2481
steps [9747/10000], loss:0.2818
steps [9748/10000], loss:0.2892
steps [9749/10000], loss:0.2714
steps [9750/10000], loss:0.2692
steps [9751/10000], loss:0.2953
steps [9752/10000], loss:0.3015
steps [9753/10000], loss:0.2689
steps [9754/10000], loss:0.2986
steps [9755/10000], loss:0.2736
steps [9

In [10]:
PPA_1 = PPA_1.cpu()
with torch.no_grad():
    _,_,vector_PPA_1 = PPA_1(torch.Tensor(vector)- mean_vector_1.cpu())

In [11]:
mean_vector_2 = vector_PPA_1.mean(0,keepdim = True).cuda()

In [12]:
PCA = autoencoder(1024,256)
PCA.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PCA.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,vector_PPA_1.shape[0] - batch_size+1)
    data_train = torch.Tensor(vector_PPA_1[pos:pos+batch_size,:]).cuda()-mean_vector_2
    # ===================forward=====================
    data_reconstruction, _,_ = PCA(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PCA.state_dict(), './dimension_reduction/PCA_1024.pth')

steps [1/10000], loss:0.3039
steps [2/10000], loss:0.2991
steps [3/10000], loss:0.3188
steps [4/10000], loss:0.2859
steps [5/10000], loss:0.2889
steps [6/10000], loss:0.2853
steps [7/10000], loss:0.3046
steps [8/10000], loss:0.2670
steps [9/10000], loss:0.2768
steps [10/10000], loss:0.2766
steps [11/10000], loss:0.2654
steps [12/10000], loss:0.3068
steps [13/10000], loss:0.3225
steps [14/10000], loss:0.2392
steps [15/10000], loss:0.2218
steps [16/10000], loss:0.2666
steps [17/10000], loss:0.2443
steps [18/10000], loss:0.2257
steps [19/10000], loss:0.2788
steps [20/10000], loss:0.2550
steps [21/10000], loss:0.2145
steps [22/10000], loss:0.2613
steps [23/10000], loss:0.2127
steps [24/10000], loss:0.2485
steps [25/10000], loss:0.2165
steps [26/10000], loss:0.2412
steps [27/10000], loss:0.2558
steps [28/10000], loss:0.2059
steps [29/10000], loss:0.2652
steps [30/10000], loss:0.2406
steps [31/10000], loss:0.2497
steps [32/10000], loss:0.2218
steps [33/10000], loss:0.2134
steps [34/10000], l

steps [339/10000], loss:0.1068
steps [340/10000], loss:0.1095
steps [341/10000], loss:0.1005
steps [342/10000], loss:0.0936
steps [343/10000], loss:0.1016
steps [344/10000], loss:0.0920
steps [345/10000], loss:0.0975
steps [346/10000], loss:0.1056
steps [347/10000], loss:0.0908
steps [348/10000], loss:0.0936
steps [349/10000], loss:0.1057
steps [350/10000], loss:0.0986
steps [351/10000], loss:0.1149
steps [352/10000], loss:0.0990
steps [353/10000], loss:0.0894
steps [354/10000], loss:0.1008
steps [355/10000], loss:0.1152
steps [356/10000], loss:0.0946
steps [357/10000], loss:0.1321
steps [358/10000], loss:0.1042
steps [359/10000], loss:0.1114
steps [360/10000], loss:0.0955
steps [361/10000], loss:0.0975
steps [362/10000], loss:0.1123
steps [363/10000], loss:0.1083
steps [364/10000], loss:0.1007
steps [365/10000], loss:0.1018
steps [366/10000], loss:0.0997
steps [367/10000], loss:0.0872
steps [368/10000], loss:0.0996
steps [369/10000], loss:0.1087
steps [370/10000], loss:0.1082
steps [3

steps [700/10000], loss:0.0943
steps [701/10000], loss:0.0952
steps [702/10000], loss:0.1201
steps [703/10000], loss:0.0870
steps [704/10000], loss:0.0939
steps [705/10000], loss:0.0931
steps [706/10000], loss:0.0910
steps [707/10000], loss:0.1076
steps [708/10000], loss:0.0972
steps [709/10000], loss:0.0971
steps [710/10000], loss:0.0895
steps [711/10000], loss:0.0885
steps [712/10000], loss:0.0895
steps [713/10000], loss:0.0938
steps [714/10000], loss:0.0912
steps [715/10000], loss:0.0954
steps [716/10000], loss:0.0881
steps [717/10000], loss:0.0880
steps [718/10000], loss:0.0935
steps [719/10000], loss:0.1123
steps [720/10000], loss:0.1082
steps [721/10000], loss:0.0910
steps [722/10000], loss:0.0978
steps [723/10000], loss:0.0988
steps [724/10000], loss:0.0949
steps [725/10000], loss:0.0941
steps [726/10000], loss:0.0942
steps [727/10000], loss:0.0898
steps [728/10000], loss:0.1002
steps [729/10000], loss:0.0924
steps [730/10000], loss:0.0965
steps [731/10000], loss:0.0854
steps [7

steps [1058/10000], loss:0.1052
steps [1059/10000], loss:0.0885
steps [1060/10000], loss:0.0948
steps [1061/10000], loss:0.0977
steps [1062/10000], loss:0.1059
steps [1063/10000], loss:0.0922
steps [1064/10000], loss:0.1025
steps [1065/10000], loss:0.1094
steps [1066/10000], loss:0.0918
steps [1067/10000], loss:0.1157
steps [1068/10000], loss:0.1080
steps [1069/10000], loss:0.1050
steps [1070/10000], loss:0.1014
steps [1071/10000], loss:0.0935
steps [1072/10000], loss:0.1025
steps [1073/10000], loss:0.1029
steps [1074/10000], loss:0.0866
steps [1075/10000], loss:0.1019
steps [1076/10000], loss:0.1014
steps [1077/10000], loss:0.0937
steps [1078/10000], loss:0.1125
steps [1079/10000], loss:0.0894
steps [1080/10000], loss:0.0866
steps [1081/10000], loss:0.1049
steps [1082/10000], loss:0.1023
steps [1083/10000], loss:0.1113
steps [1084/10000], loss:0.0904
steps [1085/10000], loss:0.0947
steps [1086/10000], loss:0.0877
steps [1087/10000], loss:0.0893
steps [1088/10000], loss:0.0959
steps [1

steps [1412/10000], loss:0.1156
steps [1413/10000], loss:0.0927
steps [1414/10000], loss:0.0902
steps [1415/10000], loss:0.0905
steps [1416/10000], loss:0.0823
steps [1417/10000], loss:0.0955
steps [1418/10000], loss:0.0961
steps [1419/10000], loss:0.1233
steps [1420/10000], loss:0.0921
steps [1421/10000], loss:0.0894
steps [1422/10000], loss:0.0976
steps [1423/10000], loss:0.0925
steps [1424/10000], loss:0.1006
steps [1425/10000], loss:0.0895
steps [1426/10000], loss:0.0927
steps [1427/10000], loss:0.1097
steps [1428/10000], loss:0.0932
steps [1429/10000], loss:0.0782
steps [1430/10000], loss:0.0934
steps [1431/10000], loss:0.0964
steps [1432/10000], loss:0.1033
steps [1433/10000], loss:0.1031
steps [1434/10000], loss:0.0943
steps [1435/10000], loss:0.0894
steps [1436/10000], loss:0.0890
steps [1437/10000], loss:0.1051
steps [1438/10000], loss:0.0943
steps [1439/10000], loss:0.0921
steps [1440/10000], loss:0.1033
steps [1441/10000], loss:0.0968
steps [1442/10000], loss:0.1039
steps [1

...

steps [8199/10000], loss:0.0994
steps [8200/10000], loss:0.0975
steps [8201/10000], loss:0.1017
steps [8202/10000], loss:0.0971
steps [8203/10000], loss:0.0931
steps [8204/10000], loss:0.1061
steps [8205/10000], loss:0.0965
steps [8206/10000], loss:0.0942
steps [8207/10000], loss:0.0818
steps [8208/10000], loss:0.0897
steps [8209/10000], loss:0.0834
steps [8210/10000], loss:0.0916
steps [8211/10000], loss:0.1001
steps [8212/10000], loss:0.0944
steps [8213/10000], loss:0.0910
steps [8214/10000], loss:0.0998
steps [8215/10000], loss:0.0853
steps [8216/10000], loss:0.1045
steps [8217/10000], loss:0.1054
steps [8218/10000], loss:0.0811
steps [8219/10000], loss:0.0921
steps [8220/10000], loss:0.0958
steps [8221/10000], loss:0.0928
steps [8222/10000], loss:0.0874
steps [8223/10000], loss:0.0931
steps [8224/10000], loss:0.0933
steps [8225/10000], loss:0.1064
steps [8226/10000], loss:0.1094
steps [8227/10000], loss:0.0913
steps [8228/10000], loss:0.0953
steps [8229/10000], loss:0.0978
steps [8

steps [8554/10000], loss:0.0975
steps [8555/10000], loss:0.0954
steps [8556/10000], loss:0.0830
steps [8557/10000], loss:0.0962
steps [8558/10000], loss:0.0975
steps [8559/10000], loss:0.0835
steps [8560/10000], loss:0.0811
steps [8561/10000], loss:0.1086
steps [8562/10000], loss:0.0946
steps [8563/10000], loss:0.1035
steps [8564/10000], loss:0.0924
steps [8565/10000], loss:0.1101
steps [8566/10000], loss:0.0880
steps [8567/10000], loss:0.0856
steps [8568/10000], loss:0.0942
steps [8569/10000], loss:0.0969
steps [8570/10000], loss:0.0884
steps [8571/10000], loss:0.0876
steps [8572/10000], loss:0.0897
steps [8573/10000], loss:0.0920
steps [8574/10000], loss:0.0823
steps [8575/10000], loss:0.0842
steps [8576/10000], loss:0.0939
steps [8577/10000], loss:0.0974
steps [8578/10000], loss:0.1052
steps [8579/10000], loss:0.1066
steps [8580/10000], loss:0.0924
steps [8581/10000], loss:0.0951
steps [8582/10000], loss:0.1261
steps [8583/10000], loss:0.1069
steps [8584/10000], loss:0.0980
steps [8

steps [8922/10000], loss:0.0838
steps [8923/10000], loss:0.1026
steps [8924/10000], loss:0.0990
steps [8925/10000], loss:0.0915
steps [8926/10000], loss:0.0937
steps [8927/10000], loss:0.0778
steps [8928/10000], loss:0.1031
steps [8929/10000], loss:0.1080
steps [8930/10000], loss:0.1006
steps [8931/10000], loss:0.1093
steps [8932/10000], loss:0.1244
steps [8933/10000], loss:0.0893
steps [8934/10000], loss:0.0925
steps [8935/10000], loss:0.0920
steps [8936/10000], loss:0.0985
steps [8937/10000], loss:0.0958
steps [8938/10000], loss:0.0850
steps [8939/10000], loss:0.0896
steps [8940/10000], loss:0.1016
steps [8941/10000], loss:0.1014
steps [8942/10000], loss:0.0968
steps [8943/10000], loss:0.0937
steps [8944/10000], loss:0.0966
steps [8945/10000], loss:0.0912
steps [8946/10000], loss:0.1059
steps [8947/10000], loss:0.1011
steps [8948/10000], loss:0.0902
steps [8949/10000], loss:0.0970
steps [8950/10000], loss:0.0964
steps [8951/10000], loss:0.0844
steps [8952/10000], loss:0.0937
steps [8

steps [9278/10000], loss:0.0870
steps [9279/10000], loss:0.0818
steps [9280/10000], loss:0.0938
steps [9281/10000], loss:0.0987
steps [9282/10000], loss:0.0892
steps [9283/10000], loss:0.1044
steps [9284/10000], loss:0.0879
steps [9285/10000], loss:0.0917
steps [9286/10000], loss:0.0963
steps [9287/10000], loss:0.0915
steps [9288/10000], loss:0.1020
steps [9289/10000], loss:0.0850
steps [9290/10000], loss:0.1011
steps [9291/10000], loss:0.0986
steps [9292/10000], loss:0.0901
steps [9293/10000], loss:0.0912
steps [9294/10000], loss:0.0945
steps [9295/10000], loss:0.0889
steps [9296/10000], loss:0.0835
steps [9297/10000], loss:0.1194
steps [9298/10000], loss:0.0892
steps [9299/10000], loss:0.0926
steps [9300/10000], loss:0.0877
steps [9301/10000], loss:0.0907
steps [9302/10000], loss:0.0890
steps [9303/10000], loss:0.0992
steps [9304/10000], loss:0.1005
steps [9305/10000], loss:0.0930
steps [9306/10000], loss:0.0830
steps [9307/10000], loss:0.0854
steps [9308/10000], loss:0.0909
steps [9

steps [9632/10000], loss:0.1005
steps [9633/10000], loss:0.0968
steps [9634/10000], loss:0.1204
steps [9635/10000], loss:0.0938
steps [9636/10000], loss:0.0843
steps [9637/10000], loss:0.0989
steps [9638/10000], loss:0.0992
steps [9639/10000], loss:0.0943
steps [9640/10000], loss:0.0976
steps [9641/10000], loss:0.0956
steps [9642/10000], loss:0.0887
steps [9643/10000], loss:0.0873
steps [9644/10000], loss:0.0928
steps [9645/10000], loss:0.0847
steps [9646/10000], loss:0.1002
steps [9647/10000], loss:0.0986
steps [9648/10000], loss:0.1193
steps [9649/10000], loss:0.1016
steps [9650/10000], loss:0.0845
steps [9651/10000], loss:0.1056
steps [9652/10000], loss:0.0940
steps [9653/10000], loss:0.0868
steps [9654/10000], loss:0.0995
steps [9655/10000], loss:0.0889
steps [9656/10000], loss:0.0934
steps [9657/10000], loss:0.0922
steps [9658/10000], loss:0.0966
steps [9659/10000], loss:0.0849
steps [9660/10000], loss:0.0995
steps [9661/10000], loss:0.0919
steps [9662/10000], loss:0.1109
steps [9

In [13]:
PCA = PCA.cpu()
with torch.no_grad():
    _,vector_PCA,_ = PCA(torch.Tensor(vector_PPA_1) - mean_vector_2.cpu())

In [14]:
mean_vector_3 = vector_PCA.mean(0,keepdim = True).cuda()

In [15]:
PPA_2 = autoencoder(256,4)
PPA_2.cuda()
criterion = nn.MSELoss()
criterion.cuda()
optimizer = torch.optim.Adam(
    PPA_2.parameters(), lr=learning_rate, weight_decay=1e-5)

for k in range(steps):
    pos = np.random.randint(0,vector_PCA.shape[0] - batch_size+1)
    data_train = torch.Tensor(vector_PCA[pos:pos+batch_size,:]).cuda()-mean_vector_3
    # ===================forward=====================
    data_reconstruction, _,_ = PPA_2(data_train)
    loss = criterion(data_train, data_reconstruction)
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    if steps%100 == 0:
        print('steps [{}/{}], loss:{:.4f}'.format(k + 1, steps, loss.item()))


torch.save(PPA_2.state_dict(), './dimension_reduction/PPA2_1024.pth')

steps [1/10000], loss:0.4075
steps [2/10000], loss:0.5093
steps [3/10000], loss:0.4760
steps [4/10000], loss:0.4489
steps [5/10000], loss:0.4418
steps [6/10000], loss:0.4099
steps [7/10000], loss:0.4118
steps [8/10000], loss:0.4185
steps [9/10000], loss:0.4019
steps [10/10000], loss:0.4416
steps [11/10000], loss:0.4238
steps [12/10000], loss:0.4454
steps [13/10000], loss:0.4275
steps [14/10000], loss:0.3973
steps [15/10000], loss:0.4270
steps [16/10000], loss:0.3981
steps [17/10000], loss:0.4036
steps [18/10000], loss:0.3772
steps [19/10000], loss:0.4236
steps [20/10000], loss:0.4448
steps [21/10000], loss:0.4705
steps [22/10000], loss:0.4048
steps [23/10000], loss:0.4269
steps [24/10000], loss:0.4327
steps [25/10000], loss:0.4161
steps [26/10000], loss:0.4438
steps [27/10000], loss:0.4742
steps [28/10000], loss:0.3752
steps [29/10000], loss:0.4417
steps [30/10000], loss:0.4152
steps [31/10000], loss:0.4164
steps [32/10000], loss:0.3752
steps [33/10000], loss:0.3653
steps [34/10000], l

steps [455/10000], loss:0.3472
steps [456/10000], loss:0.4017
steps [457/10000], loss:0.4243
steps [458/10000], loss:0.3552
steps [459/10000], loss:0.4460
steps [460/10000], loss:0.3861
steps [461/10000], loss:0.3924
steps [462/10000], loss:0.3427
steps [463/10000], loss:0.3760
steps [464/10000], loss:0.4234
steps [465/10000], loss:0.3932
steps [466/10000], loss:0.3918
steps [467/10000], loss:0.4074
steps [468/10000], loss:0.4163
steps [469/10000], loss:0.4717
steps [470/10000], loss:0.3874
steps [471/10000], loss:0.3776
steps [472/10000], loss:0.3950
steps [473/10000], loss:0.3796
steps [474/10000], loss:0.3657
steps [475/10000], loss:0.3879
steps [476/10000], loss:0.4932
steps [477/10000], loss:0.3446
steps [478/10000], loss:0.3479
steps [479/10000], loss:0.4041
steps [480/10000], loss:0.4289
steps [481/10000], loss:0.3667
steps [482/10000], loss:0.3570
steps [483/10000], loss:0.4465
steps [484/10000], loss:0.3473
steps [485/10000], loss:0.3880
steps [486/10000], loss:0.3601
steps [4

steps [905/10000], loss:0.3738
steps [906/10000], loss:0.3595
steps [907/10000], loss:0.3524
steps [908/10000], loss:0.3626
steps [909/10000], loss:0.3751
steps [910/10000], loss:0.3568
steps [911/10000], loss:0.4262
steps [912/10000], loss:0.3928
steps [913/10000], loss:0.4240
steps [914/10000], loss:0.3815
steps [915/10000], loss:0.3608
steps [916/10000], loss:0.3570
steps [917/10000], loss:0.3803
steps [918/10000], loss:0.3526
steps [919/10000], loss:0.3932
steps [920/10000], loss:0.3915
steps [921/10000], loss:0.3424
steps [922/10000], loss:0.4733
steps [923/10000], loss:0.4022
steps [924/10000], loss:0.3769
steps [925/10000], loss:0.5548
steps [926/10000], loss:0.4338
steps [927/10000], loss:0.3616
steps [928/10000], loss:0.4498
steps [929/10000], loss:0.3629
steps [930/10000], loss:0.3437
steps [931/10000], loss:0.3816
steps [932/10000], loss:0.3678
steps [933/10000], loss:0.3544
steps [934/10000], loss:0.3331
steps [935/10000], loss:0.3446
steps [936/10000], loss:0.3662
steps [9

steps [1361/10000], loss:0.3682
steps [1362/10000], loss:0.4086
steps [1363/10000], loss:0.3374
steps [1364/10000], loss:0.3889
steps [1365/10000], loss:0.3639
steps [1366/10000], loss:0.3773
steps [1367/10000], loss:0.3268
steps [1368/10000], loss:0.3806
steps [1369/10000], loss:0.3632
steps [1370/10000], loss:0.3314
steps [1371/10000], loss:0.3881
steps [1372/10000], loss:0.3695
steps [1373/10000], loss:0.4090
steps [1374/10000], loss:0.3788
steps [1375/10000], loss:0.3688
steps [1376/10000], loss:0.3794
steps [1377/10000], loss:0.3573
steps [1378/10000], loss:0.3390
steps [1379/10000], loss:0.3352
steps [1380/10000], loss:0.3387
steps [1381/10000], loss:0.3607
steps [1382/10000], loss:0.3711
steps [1383/10000], loss:0.3592
steps [1384/10000], loss:0.4425
steps [1385/10000], loss:0.3933
steps [1386/10000], loss:0.3901
steps [1387/10000], loss:0.4075
steps [1388/10000], loss:0.3673
steps [1389/10000], loss:0.3646
steps [1390/10000], loss:0.3734
steps [1391/10000], loss:0.3624
steps [1

steps [1809/10000], loss:0.4005
steps [1810/10000], loss:0.3740
steps [1811/10000], loss:0.3819
steps [1812/10000], loss:0.3830
steps [1813/10000], loss:0.3759
steps [1814/10000], loss:0.3679
steps [1815/10000], loss:0.3476
steps [1816/10000], loss:0.3620
steps [1817/10000], loss:0.3914
steps [1818/10000], loss:0.3575
steps [1819/10000], loss:0.3369
steps [1820/10000], loss:0.3447
steps [1821/10000], loss:0.4579
steps [1822/10000], loss:0.3907
steps [1823/10000], loss:0.4212
steps [1824/10000], loss:0.3654
steps [1825/10000], loss:0.4204
steps [1826/10000], loss:0.3639
steps [1827/10000], loss:0.3910
steps [1828/10000], loss:0.4089
steps [1829/10000], loss:0.3887
steps [1830/10000], loss:0.3812
steps [1831/10000], loss:0.4850
steps [1832/10000], loss:0.3847
steps [1833/10000], loss:0.3536
steps [1834/10000], loss:0.3745
steps [1835/10000], loss:0.3588
steps [1836/10000], loss:0.3849
steps [1837/10000], loss:0.3503
steps [1838/10000], loss:0.3406
steps [1839/10000], loss:0.3662
steps [1

...

steps [8138/10000], loss:0.3805
steps [8139/10000], loss:0.3799
steps [8140/10000], loss:0.3981
steps [8141/10000], loss:0.4803
steps [8142/10000], loss:0.3463
steps [8143/10000], loss:0.3810
steps [8144/10000], loss:0.3310
steps [8145/10000], loss:0.3433
steps [8146/10000], loss:0.3691
steps [8147/10000], loss:0.3473
steps [8148/10000], loss:0.3884
steps [8149/10000], loss:0.3726
steps [8150/10000], loss:0.3949
steps [8151/10000], loss:0.3540
steps [8152/10000], loss:0.3852
steps [8153/10000], loss:0.3745
steps [8154/10000], loss:0.3396
steps [8155/10000], loss:0.3423
steps [8156/10000], loss:0.3805
steps [8157/10000], loss:0.3956
steps [8158/10000], loss:0.3649
steps [8159/10000], loss:0.3932
steps [8160/10000], loss:0.3781
steps [8161/10000], loss:0.3825
steps [8162/10000], loss:0.3529
steps [8163/10000], loss:0.3910
steps [8164/10000], loss:0.3623
steps [8165/10000], loss:0.3412
steps [8166/10000], loss:0.3562
steps [8167/10000], loss:0.3838
steps [8168/10000], loss:0.4155
steps [8

steps [8585/10000], loss:0.3609
steps [8586/10000], loss:0.3712
steps [8587/10000], loss:0.3470
steps [8588/10000], loss:0.3555
steps [8589/10000], loss:0.4028
steps [8590/10000], loss:0.3615
steps [8591/10000], loss:0.3522
steps [8592/10000], loss:0.3437
steps [8593/10000], loss:0.3382
steps [8594/10000], loss:0.4025
steps [8595/10000], loss:0.3431
steps [8596/10000], loss:0.4420
steps [8597/10000], loss:0.3497
steps [8598/10000], loss:0.3372
steps [8599/10000], loss:0.3717
steps [8600/10000], loss:0.4361
steps [8601/10000], loss:0.3780
steps [8602/10000], loss:0.3439
steps [8603/10000], loss:0.3766
steps [8604/10000], loss:0.3452
steps [8605/10000], loss:0.3967
steps [8606/10000], loss:0.3623
steps [8607/10000], loss:0.3656
steps [8608/10000], loss:0.3411
steps [8609/10000], loss:0.3772
steps [8610/10000], loss:0.3768
steps [8611/10000], loss:0.3614
steps [8612/10000], loss:0.3422
steps [8613/10000], loss:0.3844
steps [8614/10000], loss:0.3921
steps [8615/10000], loss:0.3488
steps [8

steps [9038/10000], loss:0.3340
steps [9039/10000], loss:0.3447
steps [9040/10000], loss:0.3785
steps [9041/10000], loss:0.3595
steps [9042/10000], loss:0.3718
steps [9043/10000], loss:0.3625
steps [9044/10000], loss:0.3971
steps [9045/10000], loss:0.3138
steps [9046/10000], loss:0.3483
steps [9047/10000], loss:0.3527
steps [9048/10000], loss:0.4141
steps [9049/10000], loss:0.3670
steps [9050/10000], loss:0.3545
steps [9051/10000], loss:0.3997
steps [9052/10000], loss:0.3577
steps [9053/10000], loss:0.3420
steps [9054/10000], loss:0.3899
steps [9055/10000], loss:0.4808
steps [9056/10000], loss:0.3207
steps [9057/10000], loss:0.3215
steps [9058/10000], loss:0.3447
steps [9059/10000], loss:0.3387
steps [9060/10000], loss:0.3629
steps [9061/10000], loss:0.3872
steps [9062/10000], loss:0.3542
steps [9063/10000], loss:0.3622
steps [9064/10000], loss:0.3730
steps [9065/10000], loss:0.3947
steps [9066/10000], loss:0.3791
steps [9067/10000], loss:0.3199
steps [9068/10000], loss:0.4280
steps [9

steps [9497/10000], loss:0.3612
steps [9498/10000], loss:0.3831
steps [9499/10000], loss:0.4391
steps [9500/10000], loss:0.3808
steps [9501/10000], loss:0.3757
steps [9502/10000], loss:0.4100
steps [9503/10000], loss:0.3672
steps [9504/10000], loss:0.4253
steps [9505/10000], loss:0.3721
steps [9506/10000], loss:0.3707
steps [9507/10000], loss:0.3504
steps [9508/10000], loss:0.3879
steps [9509/10000], loss:0.3899
steps [9510/10000], loss:0.4083
steps [9511/10000], loss:0.4081
steps [9512/10000], loss:0.3476
steps [9513/10000], loss:0.3547
steps [9514/10000], loss:0.3541
steps [9515/10000], loss:0.3623
steps [9516/10000], loss:0.3559
steps [9517/10000], loss:0.3524
steps [9518/10000], loss:0.3943
steps [9519/10000], loss:0.3508
steps [9520/10000], loss:0.3324
steps [9521/10000], loss:0.3126
steps [9522/10000], loss:0.3743
steps [9523/10000], loss:0.3358
steps [9524/10000], loss:0.3519
steps [9525/10000], loss:0.3516
steps [9526/10000], loss:0.4063
steps [9527/10000], loss:0.3556
steps [9

steps [9941/10000], loss:0.3856
steps [9942/10000], loss:0.3655
steps [9943/10000], loss:0.3621
steps [9944/10000], loss:0.3796
steps [9945/10000], loss:0.3599
steps [9946/10000], loss:0.3401
steps [9947/10000], loss:0.3436
steps [9948/10000], loss:0.3690
steps [9949/10000], loss:0.3581
steps [9950/10000], loss:0.3631
steps [9951/10000], loss:0.4110
steps [9952/10000], loss:0.3737
steps [9953/10000], loss:0.3757
steps [9954/10000], loss:0.3876
steps [9955/10000], loss:0.3290
steps [9956/10000], loss:0.3692
steps [9957/10000], loss:0.3101
steps [9958/10000], loss:0.3452
steps [9959/10000], loss:0.3481
steps [9960/10000], loss:0.3963
steps [9961/10000], loss:0.3865
steps [9962/10000], loss:0.3369
steps [9963/10000], loss:0.3803
steps [9964/10000], loss:0.3849
steps [9965/10000], loss:0.4483
steps [9966/10000], loss:0.3631
steps [9967/10000], loss:0.3842
steps [9968/10000], loss:0.3455
steps [9969/10000], loss:0.3924
steps [9970/10000], loss:0.3319
steps [9971/10000], loss:0.3339
steps [9

In [16]:
d_r = dimension_reduction(mean_vector_1,mean_vector_2,mean_vector_3)

In [17]:
d_r.ppa1.load_state_dict(torch.load("./dimension_reduction/PPA1_1024.pth"))
d_r.pca.load_state_dict(torch.load("./dimension_reduction/PCA_1024.pth"))
d_r.ppa2.load_state_dict(torch.load("./dimension_reduction/PPA2_1024.pth"))

In [18]:
torch.save(d_r.state_dict(), './dimension_reduction/DR_1024.pth')